In [1]:
import pandas as pd
#functions
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report

from warnings import warn

from sklearn.svm import LinearSVC

from sklearn.feature_selection import SelectFromModel
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt



# Import classifiers
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import label_ranking_average_precision_score
from sklearn import tree



def multilabel_sample(y, size=1000, min_count=5, seed=None):
    """ Takes a matrix of binary labels `y` and returns
        the indices for a sample of size `size` if
        `size` > 1 or `size` * len(y) if size =< 1.
        The sample is guaranteed to have > `min_count` of
        each label.
    """
    try:
        if (np.unique(y).astype(int) != np.array([0, 1])).all():
            raise ValueError()
    except (TypeError, ValueError):
        raise ValueError('multilabel_sample only works with binary indicator matrices')

    if (y.sum(axis=0) < min_count).any():
        raise ValueError('Some classes do not have enough examples. Change min_count if necessary.')

    if size <= 1:
        size = np.floor(y.shape[0] * size)

    if y.shape[1] * min_count > size:
        msg = "Size less than number of columns * min_count, returning {} items instead of {}."
        warn(msg.format(y.shape[1] * min_count, size))
        size = y.shape[1] * min_count

    rng = np.random.RandomState(seed if seed is not None else np.random.randint(1))

    if isinstance(y, pd.DataFrame):
        choices = y.index
        y = y.values
    else:
        choices = np.arange(y.shape[0])

    sample_idxs = np.array([], dtype=choices.dtype)

    # first, guarantee > min_count of each label
    for j in range(y.shape[1]):
        label_choices = choices[y[:, j] == 1]
        label_idxs_sampled = rng.choice(label_choices, size=min_count, replace=False)
        sample_idxs = np.concatenate([label_idxs_sampled, sample_idxs])

    sample_idxs = np.unique(sample_idxs)

    # now that we have at least min_count of each, we can just random sample
    sample_count = int(size - sample_idxs.shape[0])

    # get sample_count indices from remaining choices
    remaining_choices = np.setdiff1d(choices, sample_idxs)
    remaining_sampled = rng.choice(remaining_choices,
                                   size=sample_count,
                                   replace=False)

    return np.concatenate([sample_idxs, remaining_sampled])


def multilabel_sample_dataframe(df, labels, size, min_count=5, seed=None):
    """ Takes a dataframe `df` and returns a sample of size `size` where all
        classes in the binary matrix `labels` are represented at
        least `min_count` times.
    """
    idxs = multilabel_sample(labels, size=size, min_count=min_count, seed=seed)
    return df.loc[idxs]


def multilabel_train_test_split(X, Y, size, min_count=5, seed=None):
    """ Takes a features matrix `X` and a label matrix `Y` and
        returns (X_train, X_test, Y_train, Y_test) where all
        classes in Y are represented at least `min_count` times.
    """
    index = Y.index if isinstance(Y, pd.DataFrame) else np.arange(Y.shape[0])

    test_set_idxs = multilabel_sample(Y, size=size, min_count=min_count, seed=seed)
    train_set_idxs = np.setdiff1d(index, test_set_idxs)

    test_set_mask = index.isin(test_set_idxs)
    train_set_mask = ~test_set_mask

    return (X[train_set_mask], X[test_set_mask], Y[train_set_mask], Y[test_set_mask])

In [2]:
#Selecteer goede bestand

df = pd.read_csv('TK2010_revised.csv', sep=',' ,encoding= 'latin1')
df = df[['gemeente','Partij']]

df2 = pd.read_csv('2010_all.csv',sep=',')
df2 = df2.rename(columns={"Regio's": "gemeente"})
df2 = df2[np.isfinite(df2['Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal)'])]



df3 = pd.merge(df,df2, on='gemeente')



#NUMBER = list(df3.columns[3:])

NUMBER = list(df3.columns[pd.Series(df3.columns).str.startswith('Bevolking')])




In [3]:




# Create the new DataFrame: numeric_data_only
numeric_data_only = df3[NUMBER].fillna(-1000)

X = numeric_data_only
# Get labels and convert to dummy variables: label_dummies
label_dummies = pd.get_dummies(df3['Partij'])
y = label_dummies
# Create training and test sets
X_train, X_test, y_train, y_test = multilabel_train_test_split(numeric_data_only,
                                                               label_dummies,
                                                              size=0.25,seed=25,min_count = 2)


# Instantiate the classifier: clf
clf = OneVsRestClassifier(LogisticRegression())
clf2 = tree.DecisionTreeClassifier()
svc = RandomForestClassifier()


 # Fit the classifier to the training data
clf.fit(X,y)
clf2.fit(X,y)
svc.fit(X,y)

# Print the accuracy
print("Accuracy: {}".format(clf.score(X_test, y_test)))
print("Accuracy: {}".format(svc.score(X_test, y_test)))
print("Accuracy: {}".format(clf2.score(X_test, y_test)))



# Predict the labels of the test data: y_pred
y_pred = clf2.predict(X_test)


# Generate the confusion matrix and classification report

print(classification_report(y_test, y_pred))
print(label_ranking_average_precision_score(y_test, y_pred)) 


Accuracy: 0.6731707317073171
Accuracy: 0.9975609756097561
Accuracy: 1.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        62
          1       1.00      1.00      1.00        89
          2       1.00      1.00      1.00        50
          3       1.00      1.00      1.00        13
          4       1.00      1.00      1.00         7
          5       1.00      1.00      1.00       189

avg / total       1.00      1.00      1.00       410

1.0


In [4]:
from sklearn.metrics import recall_score, precision_score
from nltk.metrics import precision, recall

holdout = pd.read_csv('2012_all.csv',index_col=0, sep=',')

holdout = holdout.rename(columns={"Regio's": 'gemeente'})

holdout = holdout[np.isfinite(holdout['Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal)'])]
holdout2 = pd.read_csv('TK2012_revised.csv', sep=',' ,encoding= 'latin1')
holdout2 = holdout2[['gemeente','Partij']]
holdout2 = holdout2.set_index('gemeente')


X2 = holdout[NUMBER].fillna(-1000)

y2 = pd.get_dummies(holdout2['Partij'])



predictions = clf2.predict(X2)


#make arrays of same length

 #Format predictions in DataFrame: prediction_df
prediction_df = pd.DataFrame(columns=pd.get_dummies(df3['Partij']).columns,
                             index=holdout['gemeente'],
                             data=predictions)



prediction_df = prediction_df.iloc[::5, :]

prediction_df['Partij2'] = prediction_df.idxmax(axis=1)


prediction_df = prediction_df[['Partij2']]


final = pd.merge(holdout2, prediction_df, left_index=True, right_index=True)


a = final.Partij.values
b = final.Partij2.values

print(classification_report(a, b))

from sklearn.metrics import accuracy_score

print(accuracy_score(a,b,normalize=True))



             precision    recall  f1-score   support

        CDA       0.03      1.00      0.06         1
         CU       0.00      0.00      0.00         1
       PVDA       0.85      0.77      0.81        66
        PVV       0.04      1.00      0.07         1
        SGP       0.86      0.75      0.80         8
         SP       0.33      1.00      0.50         1
        VVD       0.99      0.68      0.81       163

avg / total       0.93      0.71      0.80       241

0.7095435684647303


/Users/macbook/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [5]:
a = final[final.Partij == 'SGP']

a.head(50)

,Partij,Partij2
gemeente,,
Hardinxveld-Giessendam,SGP,SGP
Neder-Betuwe,SGP,SGP
Nieuw-Lekkerland,SGP,SGP
Nunspeet,SGP,CDA
Reimerswaal,SGP,SGP
Staphorst,SGP,SGP
Urk,SGP,SGP
Zwartewaterland,SGP,CDA


In [6]:
a = {name: importance for name, importance in zip(NUMBER, clf2.feature_importances_)}
  
df = pd.DataFrame(list(a.items()), columns=['name', 'importance'])

df2 = df.sort_values('importance',ascending=False)
#pd.options.display.max_rows
#pd.set_option('display.max_colwidth', -1)


df2.set_index('name')

,importance
name,
Bevolking/Bevolkingssamenstelling op 1 januari/Migratieachtergrond/Migratieachtergrond. relatief/Met migratieachtergrond/Westerse migratieachtergrond (%),0.165915
Bevolking/Bevolkingssamenstelling op 1 januari/Migratieachtergrond/Migratieachtergrond. relatief/Met migratieachtergrond/Niet-westerse migratieachtergrond/Suriname (%),0.135735
Bevolking/Particuliere huishoudens/Particuliere huishoudens. relatief/Huishoudens met kinderen (%),0.088383
Bevolking/Particuliere huishoudens/Gemiddelde huishoudensgrootte (personen per 1 huishouden),0.046427
Bevolking/Bevolkingssamenstelling op 1 januari/Bevolkingsdichtheid (aantal inwoners per km²),0.041815
Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Demografische druk/Groene druk (%),0.040219
Bevolking/Particuliere huishoudens/Particuliere huishoudens. relatief/Huishoudens zonder kinderen (%),0.035787
Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen. relatief/65 tot 80 jaar (%),0.035479
Bevolking/Bevolkingssamenstelling op 1 januari/Stedelijkheidsklasse/Niet stedelijk (aantal),0.033051


In [7]:
from sklearn.decomposition import PCA

pca = PCA(n_components=8)
pca.fit(X2,y2)




# Dump components relations with features:
z = pd.DataFrame(pca.components_,columns=X.columns,index=['CDA', 'CU', 'D66', 'GL', 'PVV', 'SGP', 'SP', 'VVD'])


z = z.transpose()





zz= z.corr().abs()


z['mean'] = z.mean(axis=1)


partij = 'CDA'

aaa = z[[partij]].sort_values(by = partij,ascending=False)

aaa.head(10)


,CDA
Bevolking/Bevolkingsontwikkeling/Bevolkingsgroei/Inwoners op 31 december (aantal),0.400605
Bevolking/Bevolkingsontwikkeling/Bevolkingsgroei/Gemiddeld aantal inwoners (aantal),0.398533
Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal),0.395860
Bevolking/Bevolkingssamenstelling op 1 januari/Burgerlijke staat/Bevolking 15 jaar of ouder/Inwoners 15 jaar of ouder (aantal),0.330985
Bevolking/Bevolkingssamenstelling op 1 januari/Stedelijkheidsklasse/Zeer sterk stedelijk (aantal),0.267783
Bevolking/Bevolkingssamenstelling op 1 januari/Burgerlijke staat/Totale bevolking/Ongehuwd (aantal),0.224252
Bevolking/Bevolkingssamenstelling op 1 januari/Migratieachtergrond/Migratieachtergrond/Nederlandse achtergrond (aantal),0.224220
Bevolking/Particuliere huishoudens/Particuliere huishoudens/Totaal aantal particuliere huishoudens (aantal),0.204872
Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Vrouwen (aantal),0.201019
Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Mannen (aantal),0.194841
